### Notebook intended to test models from Groq

In [1]:
import os
from fastapi.responses import RedirectResponse
from langserve import add_routes
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings

#### Define LLM Model

In [2]:
llm = ChatGroq(temperature=0, 
               model_name="mixtral-8x7b-32768", 
               groq_api_key="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")

In [3]:
prompt_template = """\
Use the provided context to answer the user's question. If you don't know the answer, say you don't know.

Context:
{context}

Question:
{question}"""

In [4]:
rag_prompt = ChatPromptTemplate.from_template(prompt_template)

In [5]:
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the provided context to answer the user's question. If you don't know the answer, say you don't know.\n\nContext:\n{context}\n\nQuestion:\n{question}"), additional_kwargs={})])

In [6]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [7]:
from langchain.vectorstores import FAISS

In [8]:
from langchain_cohere import CohereEmbeddings
import getpass

In [9]:
embeddings_model = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=os.environ["COHERE_API_KEY"]
)

In [10]:
faiss_index = FAISS.load_local("../langserve_index", embeddings_model, allow_dangerous_deserialization=True)
retriever = faiss_index.as_retriever()

In [11]:
faiss_index

In [12]:
entry_point_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [13]:
entry_point_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AC7A431600>, search_kwargs={}),
  question: RunnablePassthrough()
}

In [14]:
from langchain.schema import StrOutputParser

In [15]:
rag_chain = entry_point_chain | rag_prompt | llm | StrOutputParser()

In [16]:
rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AC7A431600>, search_kwargs={}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the provided context to answer the user's question. If you don't know the answer, say you don't know.\n\nContext:\n{context}\n\nQuestion:\n{question}"), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AC7FB70910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AC7FB715D0>, temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()